## Example 2 - Dependent Tasks


The following example demonstrates how to create a workflow with dependent tasks with ControlFlow.
1. Pass parameters to the flow
2. Return structured results 
3. Print dictionary of results

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import controlflow as cf
import pprint as pp

### One task with two dependent tasks.

In [2]:
@cf.flow
def analyze_text(text: str):

    # Create a parent task to represent the entire analysis
    with cf.Task(
        "Analyze the given text", 
        instructions="Include each subtask result in your result",
        result_type=dict, 
        context={"text": text}
    ) as parent_task:
        
        # Child task 1: Identify key terms
        key_terms = cf.Task(
            "Identify up to 10 key terms in the text",
            result_type=list[str]
        )

        # Child task 2: Summarize (depends on key_terms)
        summary = cf.Task(
            "Summarize the text in one sentence",
            result_type=str,
            depends_on=[key_terms]
        )

    # Run the parent task, which will automatically run all child tasks
    result = parent_task.run()
    return result

################################################################################

# Execute the flow
text = """
    ControlFlow is a Python framework designed for building agentic AI workflows. It allows developers to create workflows that delegate tasks to LLM (Large Language Model) agents, which are autonomous entities capable of making decisions and performing complex tasks. The framework emphasizes a task-centric approach, enabling the creation of discrete, observable tasks that can be assigned to specialized AI agents. These tasks can be combined into flows to orchestrate more complex behaviors.

### Key Features of ControlFlow:

- **Structured Results**: Tasks can return structured data types supported by Pydantic, not just text.
- **Custom Tools**: Developers can provide any Python function as a tool for agents to use.
- **Multi-agent Collaboration**: Multiple agents can be assigned to a task to enable collaboration.
- **User Interaction**: Agents can interact with users, allowing for dynamic input and feedback.
- **Flows**: Complex workflows can be created by running tasks with a shared context and message history.

### Benefits of Using ControlFlow:

- **Seamless Integration**: Easily integrate AI capabilities with existing Python codebases.
- **Fine-grained Control**: Maintain oversight and control over AI workflows.
- **Scalability**: Suitable for both simple scripts and complex applications.
- **Transparency**: Provides insights into AI decision-making processes.
- **Rapid Prototyping**: Facilitates quick experimentation with AI-powered features.
- **Productivity**: Focus on application logic while ControlFlow manages AI orchestration.

For more detailed information, you can visit the [ControlFlow website](https://controlflow.ai/welcome).
    """
    
result = analyze_text(text)
print(result)


Output()

{'key_terms': ['Artificial intelligence', 'Machine learning', 'Deep learning', 'Neural networks', 'Robotics', 'Natural language processing', 'AI ethics', 'Autonomous vehicles', 'Big data analytics', 'AI-driven healthcare'], 'summary': 'Artificial intelligence is revolutionizing multiple industries by enhancing capabilities and efficiencies through machine learning, deep learning, robotics, natural language processing, and improving areas such as AI ethics, autonomous vehicles, big data analytics, healthcare, and cybersecurity.'}


In [7]:
result['key_terms']

['Artificial intelligence',
 'Machine learning',
 'Deep learning',
 'Neural networks',
 'Robotics',
 'Natural language processing',
 'AI ethics',
 'Autonomous vehicles',
 'Big data analytics',
 'AI-driven healthcare']

In [9]:
pp.pprint(result['summary'])


('Artificial intelligence is revolutionizing multiple industries by enhancing '
 'capabilities and efficiencies through machine learning, deep learning, '
 'robotics, natural language processing, and improving areas such as AI '
 'ethics, autonomous vehicles, big data analytics, healthcare, and '
 'cybersecurity.')


---

## Recommendations based on a text extract.

The following demonstrates how to create 
- pass text with 3 tasks in succession. 
- a structured result with Pydantic
- pass parameters within the `context`
- print it in several different ways.

In [11]:
import controlflow as cf
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import datetime

# Define Pydantic models for structured data
class ContentMetadata(BaseModel):
    author: str
    date: datetime
    category: str
    version: Optional[str] = None

class KeyInformation(BaseModel):
    topic: str
    main_points: List[str] = Field(max_items=5)
    tone: str = Field(description="The overall tone of the content (formal, informal, technical, etc.)")
    keywords: List[str] = Field(max_items=10)

class Recommendation(BaseModel):
    action: str
    priority: str = Field(description="high, medium, or low")
    rationale: str

class AnalysisResult(BaseModel):
    metadata: ContentMetadata
    key_info: KeyInformation
    summary: str
    recommendations: List[Recommendation]

@cf.flow
def analyze_content(content: str, metadata: dict):
    # Create a parent task for the entire analysis
    with cf.Task(
        "Analyze and process content",
        instructions="Process content with subtasks and return structured results",
        result_type=AnalysisResult,
        context={"content": content, "metadata": metadata}
    ) as parent_task:
        
        # Task 1: Extract key information
        extract_info = cf.Task(
            "Extract key information from the content",
            result_type=KeyInformation,
            context={"content": content}
        )
        
        # Task 2: Generate summary (depends on extracted info)
        summary = cf.Task(
            "Generate a concise summary",
            result_type=str,
            depends_on=[extract_info]
        )
        
        # Task 3: Generate recommendations (depends on both previous tasks)
        recommendations = cf.Task(
            "Provide actionable recommendations",
            result_type=List[Recommendation],
            depends_on=[extract_info, summary]
        )
    
    # Run the parent task
    result = parent_task.run()
    return result

# Example usage
content = """
ControlFlow is a Python framework for building AI workflows. It enables developers
to create structured tasks that can be executed by AI agents. The framework
emphasizes clear objectives and validation of results.
"""

metadata = {
    "author": "John Doe",
    "date": "2024-03-20",
    "category": "Technical Documentation",
    "version": "1.0.0"
}

# Execute the flow
results = analyze_content(content, metadata)
print(results.model_dump_json(indent=2))

Output()

{
  "metadata": {
    "author": "John Doe",
    "date": "2024-03-20T00:00:00",
    "category": "Technical Documentation",
    "version": "1.0.0"
  },
  "key_info": {
    "topic": "ControlFlow: A Python Framework for AI Workflows",
    "main_points": [
      "ControlFlow is a Python framework.",
      "It is designed for building AI workflows.",
      "It allows developers to create structured tasks.",
      "Tasks can be executed by AI agents.",
      "The framework emphasizes clear objectives and result validation."
    ],
    "tone": "technical",
    "keywords": [
      "ControlFlow",
      "Python",
      "framework",
      "AI workflows",
      "structured tasks",
      "AI agents",
      "objectives",
      "validation",
      "developers",
      "execution"
    ]
  },
  "summary": "ControlFlow is a Python framework designed to build AI workflows by allowing developers to create structured tasks that can be executed by AI agents. The framework prioritizes clear objectives and resu

In [20]:
# Method 1: Print entire result as JSON (nicely formatted)
print("\n=== Complete Results as JSON ===")
print(results.model_dump_json(indent=2))

# Method 2: Print specific fields using dict format
print("\n=== Access Specific Fields (Dict Format) ===")
result_dict = results.model_dump()
print(f"Summary: {result_dict['summary']}")
print("\nKey Information:")
print(f"- Topic: {result_dict['key_info']['topic']}")
print(f"- Main Points: {result_dict['key_info']['main_points']}")

# Method 3: Access fields directly using object notation
print("\n=== Access Fields (Object Notation) ===")
print(f"Author: {results.metadata.author}")
print(f"Date: {results.metadata.date}")
print("\nRecommendations:")
for rec in results.recommendations:
    print(f"- {rec.action} (Priority: {rec.priority})")

# Method 4: Using pretty printer for nested structures
import pprint
pp = pprint.PrettyPrinter(indent=2)

print("\n=== Pretty Printed Dict Format ===")
pp.pprint(results.model_dump())

# Method 5: Print specific sections with formatting
def print_recommendations(recommendations):
    print("\n=== Detailed Recommendations ===")
    for i, rec in enumerate(recommendations, 1):
        print(f"\nRecommendation {i}:")
        print(f"Action: {rec.action}")
        print(f"Priority: {rec.priority}")
        print(f"Rationale: {rec.rationale}")

print_recommendations(results.recommendations)


=== Complete Results as JSON ===
{
  "metadata": {
    "author": "John Doe",
    "date": "2024-03-20T00:00:00",
    "category": "Technical Documentation",
    "version": "1.0.0"
  },
  "key_info": {
    "topic": "ControlFlow Framework",
    "main_points": [
      "ControlFlow is a Python framework for building AI workflows.",
      "It enables developers to create structured tasks.",
      "The framework allows execution by AI agents.",
      "Emphasizes clear objectives and validation of results."
    ],
    "tone": "technical",
    "keywords": [
      "ControlFlow",
      "Python",
      "framework",
      "AI workflows",
      "developers",
      "structured tasks",
      "AI agents",
      "objectives",
      "validation",
      "results"
    ]
  },
  "summary": "ControlFlow is a Python framework designed for building AI workflows. It allows developers to create structured tasks that can be executed by AI agents. The framework emphasizes the importance of having clear objectives a